In [1]:
import scipy.io
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential # type: ignore
from tensorflow.keras.layers import Dense, LSTM, Dropout, BatchNormalization, Bidirectional, Input, Reshape # type: ignore
from tensorflow.keras.optimizers import Adam, RMSprop  # type: ignore
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau # type: ignore
from tensorflow.keras.regularizers import l2 # type: ignore


from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
print(f"TensorFlow Version: {tf.__version__}")

# Load the .mat file
file_path = '/Users/rohitviswam/Desktop/IITM Mat file/EV_Rank_1_52_RBs_50_UEs_1000_snaps.mat'
data = scipy.io.loadmat(file_path)

# Extract the relevant data
EV_data = data['EV_re_im_split']
data = EV_data
del EV_data
print(data.shape)

TensorFlow Version: 2.16.1
(50, 1000, 832)


In [2]:
# Function to create sequences
def create_sequences(data, n_steps_in, n_steps_out):
    X, y = [], []
    for i in range(data.shape[0]):  # iterate over samples
        for j in range(data.shape[1] - n_steps_in - n_steps_out + 1):  # iterate over timesteps
            seq_x = data[i, j:j + n_steps_in]
            seq_y = data[i, j + n_steps_in:j + n_steps_in + n_steps_out]
            X.append(seq_x)
            y.append(seq_y)
    return np.array(X), np.array(y)


timesteps_in = 10

X, y = create_sequences(data, timesteps_in,5)
print(f'X shape: {X.shape}, y shape: {y.shape}')

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X shape: (49300, 10, 832), y shape: (49300, 5, 832)


# Preprocessing

In [3]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((39440, 10, 832), (9860, 10, 832), (39440, 5, 832), (9860, 5, 832))

In [4]:
# Define a function to create the advanced LSTM model with gradient clipping
def create_model(dropout_rate=0.2, lstm_units=256, dense_units=512):
    model = Sequential()
    model.add(Input(shape=(timesteps_in, 832)))
    model.add(Bidirectional(LSTM(lstm_units, return_sequences=True)))
    model.add(Dropout(dropout_rate))
    model.add(BatchNormalization())
    model.add(Bidirectional(LSTM(lstm_units)))
    model.add(Dropout(dropout_rate))
    model.add(BatchNormalization())
    model.add(Dense(dense_units, activation='relu'))
    model.add(Dropout(dropout_rate))
    model.add(BatchNormalization())
    model.add(Dense(832*5))
    model.add(Reshape((5,832)))
    
    optimizer = Adam(clipvalue=1.0)

    model.compile(optimizer=optimizer, loss='mse')
    return model

In [5]:

best_params = {

    'dropout_rate': 0.2,
    'lstm_units': 4096,
    'dense_units': 4096,
    'batch_size': 64,
    'epochs': 250,
}

# Create and train the best model
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-6)

model = create_model(
    dropout_rate=best_params['dropout_rate'],
    lstm_units=best_params['lstm_units'],
    dense_units=best_params['dense_units'],
)



In [6]:

history = model.fit(
    X_train, y_train,
    batch_size=best_params['batch_size'],
    epochs=best_params['epochs'],
    validation_split=0.2,
    callbacks=[early_stopping, reduce_lr],
    verbose=1)

Epoch 1/100
62/62 ━━━━━━━━━━━━━━━━━━━━ 54s 818ms/step - loss: 0.0909 - val_loss: 0.0146 - learning_rate: 0.0010
Epoch 2/100
62/62 ━━━━━━━━━━━━━━━━━━━━ 50s 812ms/step - loss: 0.0245 - val_loss: 0.0143 - learning_rate: 0.0010
Epoch 3/100
62/62 ━━━━━━━━━━━━━━━━━━━━ 51s 826ms/step - loss: 0.0165 - val_loss: 0.0142 - learning_rate: 0.0010
Epoch 4/100
62/62 ━━━━━━━━━━━━━━━━━━━━ 52s 833ms/step - loss: 0.0138 - val_loss: 0.0141 - learning_rate: 0.0010
Epoch 5/100
62/62 ━━━━━━━━━━━━━━━━━━━━ 51s 830ms/step - loss: 0.0122 - val_loss: 0.0136 - learning_rate: 0.0010
Epoch 6/100
62/62 ━━━━━━━━━━━━━━━━━━━━ 52s 832ms/step - loss: 0.0112 - val_loss: 0.0128 - learning_rate: 0.0010
Epoch 7/100
62/62 ━━━━━━━━━━━━━━━━━━━━ 51s 823ms/step - loss: 0.0107 - val_loss: 0.0117 - learning_rate: 0.0010
Epoch 8/100
62/62 ━━━━━━━━━━━━━━━━━━━━ 52s 831ms/step - loss: 0.0103 - val_loss: 0.0107 - learning_rate: 0.0010
Epoch 9/100
62/62 ━━━━━━━━━━━━━━━━━━━━ 51s 829ms/step - loss: 0.0101 - val_loss: 0.0100 - learning_rate:

In [7]:
test_loss = model.evaluate(X_test, y_test, verbose=1)
print(f"Test Loss: {test_loss}")

predictions = model.predict(X_test)

# Flatten y_test and predictions along the first axis
y_test_flat = y_test.reshape(-1, y_test.shape[-1])
predictions_flat = predictions.reshape(-1, predictions.shape[-1])

# Calculate metrics
mse = mean_squared_error(y_test_flat, predictions_flat)
mae = mean_absolute_error(y_test_flat, predictions_flat)
r2 = r2_score(y_test_flat, predictions_flat)

print(f'Mean Squared Error (MSE): {mse}')
print(f'Mean Absolute Error (MAE): {mae}')
print(f'R-squared (R^2): {r2}')

309/309 ━━━━━━━━━━━━━━━━━━━━ 14s 47ms/step - loss: 0.0069
Test Loss: 0.006917559076100588
309/309 ━━━━━━━━━━━━━━━━━━━━ 13s 42ms/step
Mean Squared Error (MSE): 0.0069182952803938935
Mean Absolute Error (MAE): 0.06033885367387621
R-squared (R^2): 0.5477590637154601


In [11]:
import pandas as pd
ue = 100
ts = 1
pd.DataFrame(predictions[ue][ts],y_test[ue][ts])

,0
-0.119735,-0.044312
0.088507,0.030925
0.167101,0.086936
-0.147060,-0.113126
0.073966,0.017600
...,...
-0.163436,-0.084834
-0.107874,0.007699
0.058618,0.001495
-0.078371,-0.077324
